In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from imblearn.over_sampling import SMOTE
from pycaret.classification import *

In [106]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Modelo de risco para Aviões (fatalidade + nivel do dano) 

Variáveis das ocorrencias historicas. Este modelo pode ser util para determinar politicas de segurança relacionando a idade da aeronave, marca, operador, categoria da aviação e fase da operação. 
Com o modelo desenvolvido é possivel gerar cenários para cada fase de operação de uma dada aeronava possibilitando criar melhorias nas mesmas, politicas de boas praticas, treinamento de tripulação e dentre outros.

### Leitura das bases

In [3]:
df_ocorrencias = pd.read_csv('dados/ocorrencia.csv')
df_ocorrencias.head()

,codigo_ocorrencia,classificacao,tipo,localidade,uf,pais,aerodromo,dia_ocorrencia,horario,sera_investigada,comando_investigador,status_investigacao,numero_relatorio,relatorio_publicado,dia_publicacao,quantidade_recomendacoes,aeronaves_envolvidas,saida_pista,dia_extracao
0,47965,ACIDENTE,FALHA DO MOTOR EM VOO,ARIQUEMES,RO,BRASIL,SJOG,2013-05-05,11:00:00,***,SERIPA-7,NaN,NaN,NaN,NaN,0,1,NaN,2016-07-30
1,50313,INCIDENTE GRAVE,POUSO SEM TREM,CACOAL,RO,BRASIL,SSKW,2013-11-25,12:32:00,SIM,SERIPA-7,FINALIZADA,IG-209/CENIPA/2013,1.0000,2014-04-07,0,1,NaN,2016-07-30
2,34078,ACIDENTE,PERDA DE CONTROLE NO SOLO,CEREJEIRAS,RO,BRASIL,****,2008-08-07,15:10:00,SIM,SERIPA-7,FINALIZADA,A - 517/CENIPA/2016,1.0000,2016-07-07,0,1,1.0000,2016-07-30
3,44988,ACIDENTE,POUSO LONGO,AMAJARI,RR,BRASIL,****,2011-08-11,17:00:00,SIM,SERIPA-7,FINALIZADA,A-105/CENIPA/2012,1.0000,2012-09-20,4,1,NaN,2016-07-30
4,38855,ACIDENTE,PERDA DE CONTROLE EM VOO,ACEGUÁ,RS,BRASIL,****,2009-12-28,17:30:00,SIM,SERIPA-5,ATIVA,A DEFINIR,NaN,NaN,0,1,NaN,2016-07-30


In [4]:
df_aeronaves = pd.read_csv('dados/aeronave.csv')
df_aeronaves.head()

,codigo_aeronave,codigo_ocorrencia,matricula,codigo_operador,equipamento,fabricante,modelo,tipo_motor,quantidade_motores,peso_maximo_decolagem,quantidade_assentos,ano_fabricacao,pais_registro,categoria_registro,categoria_aviacao,origem_voo,destino_voo,fase_operacao,tipo_operacao,nivel_dano,quantidade_fatalidades,dia_extracao
0,4,45602,PPGXE,241,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PISTÃO,1.0000,660,2.0000,1962.0000,BRASIL,PRI,INSTRUÇÃO,SDPW,SDPW,INDETERMINADA,INSTRUÇÃO,SUBSTANCIAL,NaN,2016-07-30
1,40,53551,PPGSZ,160,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PISTÃO,1.0000,660,2.0000,1960.0000,BRASIL,PRI,INSTRUÇÃO,SBBP,SBBP,DECOLAGEM,INSTRUÇÃO,LEVE,NaN,2016-07-30
2,118,43721,PTCMT,1232,AVIÃO,BEECH AIRCRAFT,95-B55,PISTÃO,2.0000,2310,6.0000,1966.0000,BRASIL,TPX,TÁXI AÉREO,****,****,CORRIDA APÓS POUSO,TÁXI AÉREO,SUBSTANCIAL,NaN,2016-07-30
3,130,35556,PTEQI,3992,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-721C,PISTÃO,1.0000,1633,6.0000,1979.0000,BRASIL,TPP,PARTICULAR,SNDU,SBSL,CORRIDA APÓS POUSO,PRIVADA,LEVE,NaN,2016-07-30
4,191,32579,PPVMM,4365,AVIÃO,BOEING COMPANY,737-241,JATO,2.0000,52389,117.0000,1975.0000,BRASIL,TPR,REGULAR,****,****,SUBIDA,REGULAR,NENHUM,NaN,2016-07-30


### Tratamento dados

##### Aeronaves com o campo de quantidade de motores 0

In [5]:
# tentando substituir a quantidade de motores dentro da base, buscando inicialmente pelo fabricante/modelo e em seguida substituindo pela média do fabricante e por último pela média do tipo de equipamento:
for idx, row in df_aeronaves[(df_aeronaves['quantidade_motores']==0) & (df_aeronaves['equipamento']!='PLANADOR')  | (df_aeronaves['quantidade_motores'].isna())].iterrows():
    print(f"{row['fabricante']} - {row['modelo']}")
    mean_qtd_motores =  round(df_aeronaves.loc[(df_aeronaves['quantidade_motores']!=0) & (df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']),'quantidade_motores'].mean(),0)
    if np.isnan(mean_qtd_motores):
        mean_qtd_motores =  round(df_aeronaves.loc[(df_aeronaves['quantidade_motores']!=0) & (df_aeronaves['fabricante']==row['fabricante']),'quantidade_motores'].mean(),0)
        if np.isnan(mean_qtd_motores):
            mean_qtd_motores =  round(df_aeronaves.loc[(df_aeronaves['quantidade_motores']!=0) & (df_aeronaves['equipamento']==row['equipamento']),'quantidade_motores'].mean(),0)
            
    print(mean_qtd_motores)
        
    df_aeronaves.loc[(df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']) &\
                        (df_aeronaves['codigo_ocorrencia']==row['codigo_ocorrencia']) & (df_aeronaves['codigo_aeronave']==row['codigo_aeronave']) ,
                        'quantidade_motores'] = mean_qtd_motores        

EMBRAER - EMB-195
2.0
BEECH AIRCRAFT - 100
2.0
BEECH AIRCRAFT - 58
2.0
PIPER AIRCRAFT - PA-WNE
1.0
FABRICACAO PROPRIA - ***
1.0
FABRICACAO PROPRIA - CMA
1.0
*** - SEAWIND AVIAO
1.0
PIPER AIRCRAFT - PA-34-200T
2.0
*** - PITTSS15 AVIAO
1.0
*** - P64B
1.0
*** - ***
1.0
*** - ***
1.0
*** - ***
1.0
*** - ***
1.0
*** - ***
1.0
*** - ***
1.0
*** - ***
1.0
CESSNA AIRCRAFT - A-37B
1.0
MCDONNELL DOUGLAS - MD82
3.0
CESSNA AIRCRAFT - T210L
1.0
CESSNA AIRCRAFT - N210
1.0
*** - ***
1.0
CESSNA AIRCRAFT - 210R
1.0
CESSNA AIRCRAFT - 210D
1.0
*** - ***
1.0
*** - ***
1.0
*** - ***
1.0
*** - ***
1.0
WZQ-OKECIE - PZL 106 KRUK
1.0


##### avaliando peso_maximo de decolagem zerados

In [6]:
# tentando substituir o peso máximo de decolagem zerados com valores de dentro da base, buscando inicialmente pelo fabricante/modelo e em seguida substituindo pela média do fabricante e por último pela média do tipo de equipamento:
for idx, row in df_aeronaves[(df_aeronaves['peso_maximo_decolagem']==0)].iterrows():
    print(f"{row['fabricante']} - {row['modelo']}")
    mean_peso_maximo_decolagem =  round(df_aeronaves.loc[(df_aeronaves['peso_maximo_decolagem']!=0) & (df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']),'peso_maximo_decolagem'].mean(),0)
    if np.isnan(mean_peso_maximo_decolagem):
        mean_peso_maximo_decolagem =  round(df_aeronaves.loc[(df_aeronaves['peso_maximo_decolagem']!=0) & (df_aeronaves['fabricante']==row['fabricante']),'peso_maximo_decolagem'].mean(),0)
        if np.isnan(mean_peso_maximo_decolagem):
            mean_peso_maximo_decolagem =  round(df_aeronaves.loc[(df_aeronaves['peso_maximo_decolagem']!=0) & (df_aeronaves['equipamento']==row['equipamento']),'peso_maximo_decolagem'].mean(),0)
    print(mean_peso_maximo_decolagem)
        
    df_aeronaves.loc[(df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']) &\
                        (df_aeronaves['codigo_ocorrencia']==row['codigo_ocorrencia']) & (df_aeronaves['codigo_aeronave']==row['codigo_aeronave']) ,
                        'peso_maximo_decolagem'] = mean_peso_maximo_decolagem   

FABRICACAO PROPRIA - RANS S-10-AVIAO
417.0
EMBRAER - EMB-195
4446.0
AIR TRACTOR - AT-401B
2722.0
FABRICACAO PROPRIA - TRIKE
417.0
CESSNA AIRCRAFT - R182
1406.0
FABRICACAO PROPRIA - EXPERIMENT
417.0
MAULE AIRCRAFT - ULTRALEVE
881.0
MAULE AIRCRAFT - ULTRALEVE
881.0
MAULE AIRCRAFT - ULTRALEVE
881.0
FABRICACAO PROPRIA - ***
417.0
FABRICACAO PROPRIA - CMA
417.0
FABRICACAO PROPRIA - EXPERIMENT
417.0
EXTRA-FLUGZEUGBAU - EA 300/L
6438.0
FABRICACAO PROPRIA - EXPERIMENT
417.0
*** - SEAWIND AVIAO
643.0
*** - PITTSS15 AVIAO
643.0
*** - ***
353.0
*** - ***
353.0
*** - ***
353.0
*** - ***
353.0
*** - ***
353.0
*** - ***
353.0
CESSNA AIRCRAFT - A-37B
1881.0
BOEING COMPANY - 767-3P6
79493.0
BOEING COMPANY - 747-312M
79493.0
*** - ***
353.0
*** - ***
353.0
*** - ***
353.0
PIPER AIRCRAFT - PA34
1716.0


##### Assentos zerados

In [7]:
# tentando substituir a quantidade_assentos zerados com valores de dentro da base, buscando inicialmente pelo fabricante/modelo e em seguida substituindo pela média do fabricante ou média do tipo de equipamento:
for idx, row in df_aeronaves[(df_aeronaves['quantidade_assentos']==0) | (df_aeronaves['quantidade_assentos'].isna())].iterrows():
    print(f"{row['fabricante']} - {row['modelo']}")
    mean_quantidade_assentos =  round(df_aeronaves.loc[(df_aeronaves['quantidade_assentos']!=0) & (df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']),'quantidade_assentos'].mean(),0)
    if np.isnan(mean_quantidade_assentos):
        mean_quantidade_assentos =  round(df_aeronaves.loc[(df_aeronaves['quantidade_assentos']!=0) & (df_aeronaves['fabricante']==row['fabricante']),'quantidade_assentos'].mean(),0)
        if np.isnan(mean_quantidade_assentos):
            mean_quantidade_assentos =  round(df_aeronaves.loc[(df_aeronaves['quantidade_assentos']!=0) & (df_aeronaves['equipamento']==row['equipamento']),'quantidade_assentos'].mean(),0) 
    print(mean_quantidade_assentos)

    df_aeronaves.loc[(df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']) &\
                        (df_aeronaves['codigo_ocorrencia']==row['codigo_ocorrencia']) & (df_aeronaves['codigo_aeronave']==row['codigo_aeronave']) ,
                        'quantidade_assentos'] = mean_quantidade_assentos   

FABRICACAO PROPRIA - RANS S-10-AVIAO
11.0
EMBRAER - EMB-195
10.0
BEECH AIRCRAFT - 100
6.0
BEECH AIRCRAFT - 58
6.0
PIPER AIRCRAFT - PA-23
4.0
BEECH AIRCRAFT - A35
6.0
CESSNA AIRCRAFT - 210
5.0
PIPER AIRCRAFT - PA-42
4.0
FABRICACAO PROPRIA - TRIKE
11.0
FLYER INDUSTRIA AERONAUTICA LTDA - RV6
2.0
CESSNA AIRCRAFT - 210
5.0
CESSNA AIRCRAFT - R182
4.0
BEECH AIRCRAFT - A36
6.0
FABRICACAO PROPRIA - FOX V5 SUPER
11.0
CESSNA AIRCRAFT - 210L
6.0
PIPER AIRCRAFT - PA-34-220T
6.0
PIPER AIRCRAFT - PA-WNE
4.0
CESSNA AIRCRAFT - 150M
2.0
EMBRAER - EMB-505
10.0
CESSNA AIRCRAFT - 210N
6.0
EMBRAER - EMB-505
10.0
FABRICACAO PROPRIA - EXPERIMENT
11.0
BEECH AIRCRAFT - 58
6.0
MAULE AIRCRAFT - ULTRALEVE
5.0
MAULE AIRCRAFT - ULTRALEVE
5.0
MAULE AIRCRAFT - ULTRALEVE
5.0
FABRICACAO PROPRIA - ***
11.0
CESSNA AIRCRAFT - 152
2.0
PIPER AIRCRAFT - PA-34-200
6.0
FABRICACAO PROPRIA - CMA
11.0
FABRICACAO PROPRIA - EXPERIMENT
11.0
CESSNA AIRCRAFT - 210
5.0
*** - AGUIA JA
2.0
EXTRA-FLUGZEUGBAU - EA 300/L
11.0
FABRICACAO PROP

##### analisando o ano de fabricação zerados e nulos

In [8]:
# tentando substituir o ano_fabricacao zerados com valores de dentro da base, buscando inicialmente pelo fabricante/modelo e em seguida substituindo pela média do fabricante ou média do tipo de equipamento:
for idx, row in df_aeronaves[(df_aeronaves['ano_fabricacao']==0) | (df_aeronaves['ano_fabricacao'].isna())].iterrows():
    print(f"{row['fabricante']} - {row['modelo']}")
    mean_ano_fabricacao =  round(df_aeronaves.loc[(df_aeronaves['ano_fabricacao']!=0) & (df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']),'ano_fabricacao'].mean(),0)
    if np.isnan(mean_ano_fabricacao):
        mean_ano_fabricacao =  round(df_aeronaves.loc[(df_aeronaves['ano_fabricacao']!=0) & (df_aeronaves['fabricante']==row['fabricante']),'ano_fabricacao'].mean(),0)
        if np.isnan(mean_ano_fabricacao):
            mean_ano_fabricacao =  round(df_aeronaves.loc[(df_aeronaves['ano_fabricacao']!=0) & (df_aeronaves['equipamento']==row['equipamento']),'ano_fabricacao'].mean(),0) 
    print(mean_ano_fabricacao)

    df_aeronaves.loc[(df_aeronaves['fabricante']==row['fabricante']) & (df_aeronaves['modelo']==row['modelo']) &\
                        (df_aeronaves['codigo_ocorrencia']==row['codigo_ocorrencia']) & (df_aeronaves['codigo_aeronave']==row['codigo_aeronave']) ,
                        'ano_fabricacao'] = mean_ano_fabricacao  

NEIVA INDUSTRIA AERONAUTICA - EMB-201A
1983.0
NEIVA INDUSTRIA AERONAUTICA - EMB-200A
1974.0
NEIVA INDUSTRIA AERONAUTICA - EMB-201
1976.0
FABRICACAO PROPRIA - RANS S-10-AVIAO
1984.0
NEIVA INDUSTRIA AERONAUTICA - EMB-710C
1976.0
NEIVA INDUSTRIA AERONAUTICA - EMB-201A
1983.0
HELIBRAS - HB-350B
1985.0
EMBRAER - EMB-195
1989.0
CESSNA AIRCRAFT - 210L
1974.0
CESSNA AIRCRAFT - 182C
1977.0
AGUSTA - A109S
2006.0
CESSNA AIRCRAFT - 182P
1974.0
CESSNA AIRCRAFT - 182P
1974.0
NEIVA INDUSTRIA AERONAUTICA - EMB-711ST
1982.0
BOEING COMPANY - 727-2J7
1992.0
CIA AERONAUTICA PAULISTA - CAP-4
1946.0
CESSNA AIRCRAFT - 210
1977.0
PIPER AIRCRAFT - PA-42
1979.0
NEIVA INDUSTRIA AERONAUTICA - EMB-712
1982.0
FABRICACAO PROPRIA - TRIKE
1984.0
FLYER INDUSTRIA AERONAUTICA LTDA - RV6
2005.0
PIPER AIRCRAFT - PA-28-235
1975.0
CESSNA AIRCRAFT - 210
1977.0
BEECH AIRCRAFT - C90A
1996.0
CESSNA AIRCRAFT - R182
1979.0
BEECH AIRCRAFT - A36
1981.0
FABRICACAO PROPRIA - FOX V5 SUPER
1984.0
CESSNA AIRCRAFT - 210L
1974.0
PIPER AIRC

#### Preenchendo os valores nulos
 - Os nulos da coluna quantidade_fatalidades devem ser substituidos por 0;
 - Os nulos da coluna saida_pista devem ser substituidos por 0.

In [9]:
df_aeronaves['quantidade_fatalidades'].fillna(0, inplace=True)
df_ocorrencias['saida_pista'].fillna(0, inplace=True)

### Criando classes de risco

In [123]:
# junção das tabelas aeronaves com ocorrencias
df_aeronaves_aviao = df_aeronaves[df_aeronaves['equipamento']=='AVIÃO']
df_ocorrencias_aeronave = df_aeronaves_aviao.merge(df_ocorrencias, on=['codigo_ocorrencia'], how='right')
df_ocorrencias_aeronave.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2037 entries, 0 to 2036
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   codigo_aeronave           1603 non-null   float64
 1   codigo_ocorrencia         2037 non-null   int64  
 2   matricula                 1603 non-null   object 
 3   codigo_operador           1603 non-null   float64
 4   equipamento               1603 non-null   object 
 5   fabricante                1603 non-null   object 
 6   modelo                    1603 non-null   object 
 7   tipo_motor                1603 non-null   object 
 8   quantidade_motores        1603 non-null   float64
 9   peso_maximo_decolagem     1603 non-null   float64
 10  quantidade_assentos       1603 non-null   float64
 11  ano_fabricacao            1603 non-null   float64
 12  pais_registro             1603 non-null   object 
 13  categoria_registro        1603 non-null   object 
 14  categori

In [124]:
# eliminação de targets ('quantidade_fatalidades','nivel_dano') não mapeados
df_ocorrencias_aeronave = df_ocorrencias_aeronave[(~df_ocorrencias_aeronave['quantidade_fatalidades'].isna()) & (~df_ocorrencias_aeronave['nivel_dano'].isna()) & (df_ocorrencias_aeronave['nivel_dano']!='***')]

In [125]:
targets = df_ocorrencias_aeronave[['quantidade_fatalidades','nivel_dano','classificacao']].drop_duplicates()#,	'tipo'
targets.sort_values(by=['nivel_dano','quantidade_fatalidades'])

,quantidade_fatalidades,nivel_dano,classificacao
24,0.0000,DESTRUÍDA,ACIDENTE
7,1.0000,DESTRUÍDA,ACIDENTE
60,2.0000,DESTRUÍDA,ACIDENTE
12,3.0000,DESTRUÍDA,ACIDENTE
62,4.0000,DESTRUÍDA,ACIDENTE
211,5.0000,DESTRUÍDA,ACIDENTE
349,6.0000,DESTRUÍDA,ACIDENTE
368,7.0000,DESTRUÍDA,ACIDENTE
1778,8.0000,DESTRUÍDA,ACIDENTE
1630,14.0000,DESTRUÍDA,ACIDENTE


In [126]:
def categoria_map(qtd_fatalidade, nivel_dado):
    # if qtd_fatalidade>20:
    #     return '5 - MUITO ALTO'
    if qtd_fatalidade>2 and nivel_dado=='DESTRUÍDA':
        return '3 - ALTO'
    if qtd_fatalidade<=2 and nivel_dado=='DESTRUÍDA':
        return '2 - MÉDIO ALTO'
    if qtd_fatalidade>0 and nivel_dado=='SUBSTANCIAL' or (nivel_dado=='NENHUM' or nivel_dado=='LEVE' and qtd_fatalidade!=0):
        return '1 - MÉDIO'
    if nivel_dado=='NENHUM' or nivel_dado=='LEVE' or nivel_dado=='SUBSTANCIAL' and qtd_fatalidade==0 :
        return '0 - BAIXO'
    
targets['classe'] = targets.apply(lambda x: categoria_map(x['quantidade_fatalidades'], x['nivel_dano']), axis=1)
targets.sort_values(by=['nivel_dano','quantidade_fatalidades'])

,quantidade_fatalidades,nivel_dano,classificacao,classe
24,0.0000,DESTRUÍDA,ACIDENTE,2 - MÉDIO ALTO
7,1.0000,DESTRUÍDA,ACIDENTE,2 - MÉDIO ALTO
60,2.0000,DESTRUÍDA,ACIDENTE,2 - MÉDIO ALTO
12,3.0000,DESTRUÍDA,ACIDENTE,3 - ALTO
62,4.0000,DESTRUÍDA,ACIDENTE,3 - ALTO
211,5.0000,DESTRUÍDA,ACIDENTE,3 - ALTO
349,6.0000,DESTRUÍDA,ACIDENTE,3 - ALTO
368,7.0000,DESTRUÍDA,ACIDENTE,3 - ALTO
1778,8.0000,DESTRUÍDA,ACIDENTE,3 - ALTO
1630,14.0000,DESTRUÍDA,ACIDENTE,3 - ALTO


In [127]:
df_ocorrencias_aeronave = df_ocorrencias_aeronave.merge(targets,how='left')
df_ocorrencias_aeronave['classe'].value_counts()

0 - BAIXO         1091
1 - MÉDIO          203
2 - MÉDIO ALTO     194
3 - ALTO            58
Name: classe, dtype: int64

Quando o conjunto de dados de treinamento tem uma distribuição desigual da classe de destino, ela pode ser corrigida usando o parâmetro fix_imbalance na configuração. Quando definido como True, SMOTE (Synthetic Minority Over-sampling Technique) é usado como um método padrão para reamostragem.

### Criando modelo de classificação de risco

In [128]:
df_ocorrencias_aeronave

,codigo_aeronave,codigo_ocorrencia,matricula,codigo_operador,equipamento,fabricante,modelo,tipo_motor,quantidade_motores,peso_maximo_decolagem,...,comando_investigador,status_investigacao,numero_relatorio,relatorio_publicado,dia_publicacao,quantidade_recomendacoes,aeronaves_envolvidas,saida_pista,dia_extracao_y,classe
0,15062.0000,50313,PTRDP,4368.0000,AVIÃO,EMBRAER,EMB-810C,PISTÃO,2.0000,2073.0000,...,SERIPA-7,FINALIZADA,IG-209/CENIPA/2013,1.0000,2014-04-07,0,1,0.0000,2016-07-30,0 - BAIXO
1,3140.0000,34078,PPEHX,866.0000,AVIÃO,EMBRAER,EMB-820C NAVAJO,PISTÃO,2.0000,3175.0000,...,SERIPA-7,FINALIZADA,A - 517/CENIPA/2016,1.0000,2016-07-07,0,1,1.0000,2016-07-30,0 - BAIXO
2,12615.0000,44988,PTXCG,4468.0000,AVIÃO,CESSNA AIRCRAFT,U206G,PISTÃO,1.0000,1633.0000,...,SERIPA-7,FINALIZADA,A-105/CENIPA/2012,1.0000,2012-09-20,4,1,0.0000,2016-07-30,0 - BAIXO
3,10479.0000,38855,PRSUB,345.0000,AVIÃO,CESSNA AIRCRAFT,A188B,PISTÃO,1.0000,1812.0000,...,SERIPA-5,ATIVA,A DEFINIR,NaN,NaN,0,1,0.0000,2016-07-30,1 - MÉDIO
4,13405.0000,46633,PPZGV,3992.0000,AVIÃO,IVAN MOLCHAN,GV-2,PISTÃO,1.0000,430.0000,...,SERIPA-6,NaN,NaN,NaN,NaN,0,1,0.0000,2016-07-30,0 - BAIXO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,9619.0000,34081,PTOYW,3992.0000,AVIÃO,PIPER AIRCRAFT,PA-32R-301,PISTÃO,1.0000,1633.0000,...,CENIPA,NaN,NaN,NaN,NaN,0,1,0.0000,2016-07-30,0 - BAIXO
1542,34562.0000,36116,N526MD,5363.0000,AVIÃO,MCDONNELL DOUGLAS,DC-10-30,PISTÃO,3.0000,259455.0000,...,CENIPA,NaN,NaN,NaN,NaN,0,1,0.0000,2016-07-30,0 - BAIXO
1543,9574.0000,33715,PPVQY,5106.0000,AVIÃO,MCDONNELL DOUGLAS,DC-10-30F,JATO,3.0000,259455.0000,...,CENIPA,NaN,NaN,NaN,NaN,0,1,0.0000,2016-07-30,0 - BAIXO
1544,9416.0000,49474,PTMVL,4834.0000,AVIÃO,AIRBUS INDUSTRIE,A330-203,JATO,2.0000,233000.0000,...,CENIPA,ATIVA,A-158/CENIPA/2013,NaN,NaN,0,1,0.0000,2016-07-30,1 - MÉDIO


In [129]:
features=['codigo_operador', 'fabricante', 'modelo', 'tipo_motor', 
          'quantidade_motores', 'peso_maximo_decolagem', 
          'ano_fabricacao','quantidade_assentos', 'origem_voo',	'destino_voo',
          'categoria_aviacao',	'fase_operacao']
s = setup(df_ocorrencias_aeronave[features+['classe']], target = 'classe', 
          fix_imbalance = True, 
          fix_imbalance_method = SMOTE(random_state=42, k_neighbors=5, sampling_strategy='not majority', n_jobs=-1),
          fold=5,
          session_id=123)

In [130]:
pull()

,Description,Value
0,Session id,123
1,Target,classe
2,Target type,Multiclass
3,Target mapping,"0 - BAIXO: 0, 1 - MÉDIO: 1, 2 - MÉDIO ALTO: 2,..."
4,Original data shape,"(1546, 13)"
5,Transformed data shape,"(3516, 49)"
6,Transformed train set shape,"(3052, 49)"
7,Transformed test set shape,"(464, 49)"
8,Numeric features,5
9,Categorical features,7


In [131]:
best = compare_models(include=['dt','et','rf'],#
                      sort='Recall', 
                      n_select=3,
                      turbo=True)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:04:04
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Decision Tree Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.6562,0.6916,0.6562,0.6215,0.6352,0.1915,0.1946,0.7260
rf,Random Forest Classifier,0.5500,0.6576,0.5500,0.5976,0.5528,0.1094,0.1151,0.6740
dt,Decision Tree Classifier,0.3763,0.5408,0.3763,0.5818,0.4286,0.0609,0.0719,1.0440


In [132]:
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.6562,0.6916,0.6562,0.6215,0.6352,0.1915,0.1946,0.7260
rf,Random Forest Classifier,0.5500,0.6576,0.5500,0.5976,0.5528,0.1094,0.1151,0.6740
dt,Decision Tree Classifier,0.3763,0.5408,0.3763,0.5818,0.4286,0.0609,0.0719,1.0440


In [144]:
tuned_model, tuner = tune_model(best[0],n_iter=10, optimize='Recall',search_library = 'scikit-optimize',search_algorithm='bayesian', choose_better = True, return_tuner=True)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [145]:
pull()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5806,0.7093,0.5806,0.6750,0.6170,0.2454,0.2552
1,0.6037,0.7337,0.6037,0.7157,0.6461,0.2883,0.3044
2,0.5278,0.6523,0.5278,0.6272,0.5691,0.1328,0.1393
3,0.6157,0.7342,0.6157,0.6900,0.6455,0.2833,0.2909
4,0.5694,0.7025,0.5694,0.6935,0.6150,0.2586,0.2748
Mean,0.5795,0.7064,0.5795,0.6803,0.6185,0.2417,0.2529
Std,0.0306,0.0299,0.0306,0.0296,0.0281,0.0567,0.0591


In [134]:
# blender = blend_models(best)

In [135]:
predict_model(tuned_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.6509,0.6747,0,0,0,0.1268,0.1324


,codigo_operador,fabricante,modelo,tipo_motor,quantidade_motores,peso_maximo_decolagem,ano_fabricacao,quantidade_assentos,origem_voo,destino_voo,categoria_aviacao,fase_operacao,classe,prediction_label,prediction_score
1082,866.0000,EMBRAER,EMB-820C NAVAJO,PISTÃO,2.0000,3175.0000,1981.0000,8.0000,SWKK,SWRL,TÁXI AÉREO,CORRIDA APÓS POUSO,0 - BAIXO,0 - BAIXO,0.8600
1083,3992.0000,CESSNA AIRCRAFT,310R,PISTÃO,2.0000,2495.0000,1979.0000,6.0000,SSCP,SDAM,PARTICULAR,POUSO,0 - BAIXO,0 - BAIXO,0.9900
1084,3992.0000,PIPER AIRCRAFT,PA-34-200,PISTÃO,2.0000,1905.0000,1973.0000,7.0000,SWFN,SWBR,PARTICULAR,POUSO,0 - BAIXO,0 - BAIXO,0.9200
1085,3992.0000,CIRRUS DESIGN,SR22,PISTÃO,1.0000,1542.0000,2007.0000,4.0000,SBSV,SNBO,PARTICULAR,POUSO,0 - BAIXO,0 - BAIXO,0.9700
1086,187.0000,AERO BOERO,AB-115,PISTÃO,1.0000,770.0000,1991.0000,2.0000,****,****,ESPECIALIZADA,POUSO,1 - MÉDIO,0 - BAIXO,0.9500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,4088.0000,NEIVA INDUSTRIA AERONAUTICA,EMB-721C,PISTÃO,1.0000,1633.0000,1977.0000,7.0000,SDOW,****,TÁXI AÉREO,CRUZEIRO,1 - MÉDIO,0 - BAIXO,0.5600
1542,383.0000,EMBRAER,EMB-810C,PISTÃO,2.0000,2073.0000,1979.0000,7.0000,SSCL,SIFH,TÁXI AÉREO,CORRIDA APÓS POUSO,0 - BAIXO,0 - BAIXO,0.8500
1543,3992.0000,NEIVA INDUSTRIA AERONAUTICA,EMB-202,PISTÃO,1.0000,1550.0000,1998.0000,1.0000,****,****,PARTICULAR,DECOLAGEM,2 - MÉDIO ALTO,0 - BAIXO,0.8900
1544,355.0000,NEIVA INDUSTRIA AERONAUTICA,EMB-201A,PISTÃO,1.0000,1800.0000,1984.0000,1.0000,****,****,AGRÍCOLA,ESPECIALIZADA,0 - BAIXO,0 - BAIXO,0.7700


In [136]:
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.6509,0.6747,0,0,0,0.1268,0.1324


In [137]:
# dashboard(best[0])

In [138]:
evaluate_model(best[0])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [139]:
res = predict_model(best[0])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.6509,0.6747,0,0,0,0.1268,0.1324


In [140]:
res

,codigo_operador,fabricante,modelo,tipo_motor,quantidade_motores,peso_maximo_decolagem,ano_fabricacao,quantidade_assentos,origem_voo,destino_voo,categoria_aviacao,fase_operacao,classe,prediction_label,prediction_score
1082,866.0000,EMBRAER,EMB-820C NAVAJO,PISTÃO,2.0000,3175.0000,1981.0000,8.0000,SWKK,SWRL,TÁXI AÉREO,CORRIDA APÓS POUSO,0 - BAIXO,0 - BAIXO,0.8600
1083,3992.0000,CESSNA AIRCRAFT,310R,PISTÃO,2.0000,2495.0000,1979.0000,6.0000,SSCP,SDAM,PARTICULAR,POUSO,0 - BAIXO,0 - BAIXO,0.9900
1084,3992.0000,PIPER AIRCRAFT,PA-34-200,PISTÃO,2.0000,1905.0000,1973.0000,7.0000,SWFN,SWBR,PARTICULAR,POUSO,0 - BAIXO,0 - BAIXO,0.9200
1085,3992.0000,CIRRUS DESIGN,SR22,PISTÃO,1.0000,1542.0000,2007.0000,4.0000,SBSV,SNBO,PARTICULAR,POUSO,0 - BAIXO,0 - BAIXO,0.9700
1086,187.0000,AERO BOERO,AB-115,PISTÃO,1.0000,770.0000,1991.0000,2.0000,****,****,ESPECIALIZADA,POUSO,1 - MÉDIO,0 - BAIXO,0.9500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,4088.0000,NEIVA INDUSTRIA AERONAUTICA,EMB-721C,PISTÃO,1.0000,1633.0000,1977.0000,7.0000,SDOW,****,TÁXI AÉREO,CRUZEIRO,1 - MÉDIO,0 - BAIXO,0.5600
1542,383.0000,EMBRAER,EMB-810C,PISTÃO,2.0000,2073.0000,1979.0000,7.0000,SSCL,SIFH,TÁXI AÉREO,CORRIDA APÓS POUSO,0 - BAIXO,0 - BAIXO,0.8500
1543,3992.0000,NEIVA INDUSTRIA AERONAUTICA,EMB-202,PISTÃO,1.0000,1550.0000,1998.0000,1.0000,****,****,PARTICULAR,DECOLAGEM,2 - MÉDIO ALTO,0 - BAIXO,0.8900
1544,355.0000,NEIVA INDUSTRIA AERONAUTICA,EMB-201A,PISTÃO,1.0000,1800.0000,1984.0000,1.0000,****,****,AGRÍCOLA,ESPECIALIZADA,0 - BAIXO,0 - BAIXO,0.7700


In [141]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

print(accuracy_score(res['classe'], res['prediction_label']))
print(f1_score(res['classe'], res['prediction_label'], average='macro'))
print(precision_score(res['classe'], res['prediction_label'], average="macro"))
print(recall_score(res['classe'], res['prediction_label'], average="macro"))

0.6508620689655172
0.3261248991121872
0.3493069237264284
0.32230538312058543


In [142]:
print(classification_report(res['classe'], res['prediction_label']))

                precision    recall  f1-score   support

     0 - BAIXO       0.74      0.86      0.79       328
     1 - MÉDIO       0.29      0.25      0.27        61
2 - MÉDIO ALTO       0.18      0.07      0.10        58
      3 - ALTO       0.18      0.12      0.14        17

      accuracy                           0.65       464
     macro avg       0.35      0.32      0.33       464
  weighted avg       0.59      0.65      0.61       464

